[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/markbozsoki/ColabNotebooks/blob/main/Animal_Shelter_Analytics.ipynb)
[![Open in GitHub](https://badgen.net/badge/icon/Open%20in%20GitHub?icon=github&label)](https://github.com/markbozsoki/ColabNotebooks/blob/main/Animal_Shelter_Analytics.ipynb)

<h1>Állatmenhely örökbefogadási ráta és prognózis<h1>

###Adatállomány:
_Animal Shelter Analytics_

[![kaggle](https://img.shields.io/badge/Kaggle-20BEFF?style=for-the-badge&logo=Kaggle&logoColor=white)](https://www.kaggle.com/datasets/jackdaoud/animal-shelter-analytics)
[![website](https://badgen.net/badge/website/Austin%20Animal%20Center)](https://www.austintexas.gov/austin-animal-center)

https://data.austintexas.gov/browse?City-of-Austin_Department-=Animal+Services

https://marketplace.animalsheltering.org/sites/default/files/webform/animal-care-expo-2019-handouts/Saving%20by%20the%20numbers-scarlett-jan.pdf

In [1]:
# DATA ACCESS

print("Initializing Kaggle...")
!pip install -q kaggle
!mkdir -p ~/.kaggle

print("\nInitializing Kaggle API access...")
!pip install --upgrade --no-cache-dir -q gdown
!gdown 1m1sgCgeHHtjeeTtSKjZmFg-CRnuEIEHL -q --output ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

print("")
!kaggle datasets download -d jackdaoud/animal-shelter-analytics

print("\nUnzipping mars-landmarks.zip to /content/animal-shelter-analytics")
!unzip -q -o animal-shelter-analytics.zip -d animal-shelter-analytics

print("\nRemoving animal-shelter-analytics.zip")
!rm animal-shelter-analytics.zip

print("\nDone!")

Initializing Kaggle...

Initializing Kaggle API access...
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done

 65% 5.00M/7.67M [00:00<00:00, 28.5MB/s]
100% 7.67M/7.67M [00:00<00:00, 39.5MB/s]

Unzipping mars-landmarks.zip to /content/animal-shelter-analytics

Removing animal-shelter-analytics.zip

Done!


---

In [2]:
# IMPORTS

import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [3]:
# CSV DATA

intakes = pd.read_csv("animal-shelter-analytics/Austin_Animal_Center_Intakes.csv")
outcomes = pd.read_csv("animal-shelter-analytics/Austin_Animal_Center_Outcomes.csv")
stray_map = pd.read_csv("animal-shelter-analytics/Austin_Animal_Center_Stray_Map.csv")

In [4]:
intakes

,Animal ID,Name,DateTime,MonthYear,Found Location,Intake Type,Intake Condition,Animal Type,Sex upon Intake,Age upon Intake,Breed,Color
0,A786884,*Brock,01/03/2019 04:19:00 PM,01/03/2019 04:19:00 PM,2501 Magin Meadow Dr in Austin (TX),Stray,Normal,Dog,Neutered Male,2 years,Beagle Mix,Tricolor
1,A706918,Belle,07/05/2015 12:59:00 PM,07/05/2015 12:59:00 PM,9409 Bluegrass Dr in Austin (TX),Stray,Normal,Dog,Spayed Female,8 years,English Springer Spaniel,White/Liver
2,A724273,Runster,04/14/2016 06:43:00 PM,04/14/2016 06:43:00 PM,2818 Palomino Trail in Austin (TX),Stray,Normal,Dog,Intact Male,11 months,Basenji Mix,Sable/White
3,A665644,NaN,10/21/2013 07:59:00 AM,10/21/2013 07:59:00 AM,Austin (TX),Stray,Sick,Cat,Intact Female,4 weeks,Domestic Shorthair Mix,Calico
4,A682524,Rio,06/29/2014 10:38:00 AM,06/29/2014 10:38:00 AM,800 Grove Blvd in Austin (TX),Stray,Normal,Dog,Neutered Male,4 years,Doberman Pinsch/Australian Cattle Dog,Tan/Gray
...,...,...,...,...,...,...,...,...,...,...,...,...
124115,A828158,Odin,01/08/2021 07:01:00 PM,01/08/2021 07:01:00 PM,2224 E Stassney in Austin (TX),Stray,Normal,Dog,Intact Male,10 months,Rottweiler Mix,Black/Tan
124116,A830183,NaN,03/03/2021 05:12:00 PM,03/03/2021 05:12:00 PM,6802 Bryonwood Drive in Austin (TX),Stray,Medical,Dog,Intact Female,1 month,Chihuahua Shorthair,White
124117,A830180,Gigi,03/03/2021 04:31:00 PM,03/03/2021 04:31:00 PM,Austin (TX),Owner Surrender,Normal,Dog,Intact Female,9 years,Australian Cattle Dog/Belgian Malinois,Brown Brindle/White
124118,A830171,NaN,03/03/2021 05:13:00 PM,03/03/2021 05:13:00 PM,1310 West Howard Lane in Austin (TX),Stray,Normal,Dog,Intact Female,1 year,Plott Hound Mix,Brown Brindle/White


In [5]:
outcomes

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color
0,A794011,Chunk,05/08/2019 06:20:00 PM,05/08/2019 06:20:00 PM,05/02/2017,Rto-Adopt,NaN,Cat,Neutered Male,2 years,Domestic Shorthair Mix,Brown Tabby/White
1,A776359,Gizmo,07/18/2018 04:02:00 PM,07/18/2018 04:02:00 PM,07/12/2017,Adoption,NaN,Dog,Neutered Male,1 year,Chihuahua Shorthair Mix,White/Brown
2,A821648,NaN,08/16/2020 11:38:00 AM,08/16/2020 11:38:00 AM,08/16/2019,Euthanasia,NaN,Other,Unknown,1 year,Raccoon,Gray
3,A720371,Moose,02/13/2016 05:59:00 PM,02/13/2016 05:59:00 PM,10/08/2015,Adoption,NaN,Dog,Neutered Male,4 months,Anatol Shepherd/Labrador Retriever,Buff
4,A674754,NaN,03/18/2014 11:47:00 AM,03/18/2014 11:47:00 AM,03/12/2014,Transfer,Partner,Cat,Intact Male,6 days,Domestic Shorthair Mix,Orange Tabby
...,...,...,...,...,...,...,...,...,...,...,...,...
124486,A828810,*Rory,03/03/2021 04:20:00 PM,03/03/2021 04:20:00 PM,03/26/2020,Adoption,NaN,Dog,Spayed Female,11 months,Chihuahua Shorthair/American Staffordshire Ter...,Brown/Tan
124487,A791377,Bolt,12/28/2020 03:49:00 PM,12/28/2020 03:49:00 PM,01/14/2019,Return to Owner,NaN,Dog,Neutered Male,1 year,Siberian Husky Mix,White
124488,A829611,Chief,03/03/2021 04:59:00 PM,03/03/2021 04:59:00 PM,02/17/2012,Rto-Adopt,NaN,Cat,Neutered Male,9 years,Domestic Shorthair,Orange Tabby
124489,A829964,NaN,03/03/2021 05:57:00 PM,03/03/2021 05:57:00 PM,11/27/2020,Adoption,NaN,Dog,Neutered Male,3 months,Labrador Retriever,Cream


In [6]:
stray_map

,Animal ID,Found Location,At AAC,Intake Date,Type,Looks Like,Color,Sex,Age,Image Link
0,A814119,11800 GREEN GROVE DRIVE\nAUSTIN 78725\n(30.221...,Yes (come to the shelter),02/19/2021,Dog,Pit Bull,Brown Brindle,Neutered Male,3 years,http://www.petharbor.com/pet.asp?uaid=ASTN.A81...
1,A829667,9807 MARLBOROUGH DRIVE\nAUSTIN 78753\n(30.3573...,Yes (come to the shelter),02/20/2021,Dog,Anatolian Shepherd Mix,Yellow,Intact Male,2 years,http://www.petharbor.com/pet.asp?uaid=ASTN.A82...
2,A829669,12501 TECH RIDGE BOULEVARD\nAUSTIN 78753\n(30....,Yes (come to the shelter),02/20/2021,Dog,Labrador Retriever Mix,Tan/White,Spayed Female,2 years,http://www.petharbor.com/pet.asp?uaid=ASTN.A82...
3,A829713,10605 OLD LOCKHART ROAD\nAUSTIN 78747\n(30.128...,Yes (come to the shelter),02/21/2021,Dog,Doberman Pinsch,Black/Tan,Intact Female,6 months,http://www.petharbor.com/pet.asp?uaid=ASTN.A82...
4,A829659,"14204 JACOBSON ROAD\nAUSTIN 78617\n(30.137238,...",Yes (come to the shelter),02/19/2021,Dog,Labrador Retriever,Black/White,Intact Female,5 months,http://www.petharbor.com/pet.asp?uaid=ASTN.A82...
5,A829660,"14204 JACOBSON ROAD\nAUSTIN 78617\n(30.137238,...",Yes (come to the shelter),02/19/2021,Dog,Carolina Dog,Yellow Brindle/White,Intact Female,5 months,http://www.petharbor.com/pet.asp?uaid=ASTN.A82...
6,A829661,"9604 CARSON CREEK\nAUSTIN 78617\n(30.224098, -...",Yes (come to the shelter),02/19/2021,Cat,Domestic Shorthair,Orange Tabby,Intact Male,1 year,http://www.petharbor.com/pet.asp?uaid=ASTN.A82...
7,A829664,"DEL VALLE AUSTIN 78719\n(30.143087, -97.673385)",Yes (come to the shelter),02/20/2021,Dog,Labrador Retriever,Brown/Black,Intact Male,2 years,http://www.petharbor.com/pet.asp?uaid=ASTN.A82...
8,A829672,"AUSTIN 78744\n(30.186349, -97.738673)",Yes (come to the shelter),02/20/2021,Dog,Doberman Pinsch,Black/Tan,Intact Female,6 months,http://www.petharbor.com/pet.asp?uaid=ASTN.A82...
9,A716730,"4434 FRONTIER TRAIL\nAUSTIN 78745\n(30.23124, ...",Yes (come to the shelter),02/22/2021,Cat,Domestic Shorthair Mix,Brown Tabby,Spayed Female,5 years,http://www.petharbor.com/pet.asp?uaid=ASTN.A71...
